# Design for Co-Factor Swapping

In [1]:
from cameo.strain_design import CofactorSwapOptimization

ImportError: cannot import name 'CofactorSwapOptimization'

## Setup Model

In [ ]:
from cobra.io import read_sbml_model, write_sbml_model
from cobra import Reaction, Metabolite

In [ ]:
# Loading of the model
model = read_sbml_model('data/yeastGEM_het.xml')

In [ ]:
with model:
    print(model.slim_optimize())
    model.objective = model.reactions.R07215
    print(model.slim_optimize())

An updated and thus slightly larger model is also available.

In [ ]:
model

In [ ]:
model.solver = "glpk"

In [ ]:
biomass = model.reactions.r_2111

It is probably desirable to account for a growth coupled design. Ideally the value chosen here should correspond to the region chosen in the production envelope.

In [ ]:
biomass.lower_bound = 0.1

In [ ]:
target = model.metabolites.get_by_id("7-Dehydrocholesterol_c")

In [ ]:
demand = model.reactions.get_by_id("DM_7-Dehydrocholesterol_c")

In [ ]:
model.objective = demand

In [ ]:
from cameo.strain_design import product_yield

In [ ]:
prod_yield = product_yield(model.reactions.R07215, model.reactions.r_1714)


By default this method looks to replace NADH and NADPH. You can set different co-factor pairs via the parameter `cofactor_id_swaps=([from], [to])` which requires the metabolite identifiers in the model of the pairs that you want to swap. Please look at the documentation for more info.

In [ ]:
CofactorSwapOptimization?

In [ ]:
# Since our model uses other names for the co-factors, here are the names in our model:

# NADH  = s_1203[c]
# NAD+  = s_1198[c]

# NADPH    = s_1212[c]
# NADP+    = s_1207[c]

In [ ]:
py = product_yield(model.reactions.R07215, model.reactions.r_1714)
optswap = CofactorSwapOptimization(model=model, objective_function=py)

In [ ]:
for metabolite in model.metabolites.query('NADH','name'):
    print(metabolite.id)

In [ ]:
# Swap NADPH to NADH
#                                                                                            cofactor_id_swaps=(['nad_c', 'nadh_c'], ['nadp_c', 'nadph_c']),
swap_opt = CofactorSwapOptimization(model=model, objective_function = prod_yield, plot=True, cofactor_id_swaps=(['s_1198[c]', 's_1203[c]'], ['s_1207[c]', 's_1212[c]']))

In [ ]:
%%time
result = swap_opt.run(max_size=5)

In [ ]:
result

In [ ]:
with model:
    print(model.slim_optimize())
    model.objective = model.reactions.R07215
    print(model.slim_optimize())

In [ ]:
with model:
    #model.objective = model.reactions.R07215
    sol = model.optimize()
    print(sol.objective_value)